# Laboratorium 7
### Regex

In [1]:
import numpy as np

In [2]:
def trans(pattern, alf):
    mov = [{} for i in range(len(pattern)+1)]
    for i in alf:
        mov[0][i] = 0
    mov[0][pattern[0]] = 1
    lps = 0
    for i in range(1, len(pattern)+1):
        for x in alf:
            mov[i][x] = mov[lps][x]
        if i < len(pattern):
            mov[i][pattern[i]] = i+1
            lps = mov[lps][pattern[i]]
    return mov


def auto(source, pattern, alf=set(), mov=[]):
    if len(alf)==0:
        alf = set(np.unique([x for x in pattern]))
    if len(mov)==0:
        mov = trans(pattern, alf)
    q = 0
    ans = []
    for s in range(0, len(source)):
        if source[s] not in alf:
            q = 0
        else:
            q = mov[q][source[s]]
            if q == len(mov)-1:
                ans += [s+1-q]
    return ans

Wykorzystując ten kod, z laboratorium 1 na automat skończony wyszukujący wzorzec w tekście, zmienimy go by Matchował - sprawdzał czy cały napis pasuje do wyrażenia.

1. Zaimplementuj algorytm wyszukiwania wyrażeń regularnych. Wyrażenie może zawierać:
    - litery, cyfry i spacje - traktować literalnie (sine qua non)
    - kropki - reprezentuje dowolny znak (1 pkt)
    - operatory: gwiazdkę - 0 lub więcej powtórzeń poprzedniego symbolu, plus - 1 lub więcej powtórzeń, pytajnik - 0 lub 1 powtórzenie (1.5 pkt.)
    - nawiasy - na potrzeby operatorów gwiazdki, plusa i pytajnika zawartość nawiasów jest pojedynczym symbolem; nawiasy mogą być zagnieżdżone; nawiasy po których nie występuje żaden z wymienionych operatorów nie mają skutku (1 pkt)
    - klasy znaków (albo dowolna liczba znaków wymieniona w nawiasach kwadratowych, albo oznaczenie klasy, typu \d) (1.5 pkt.)

Możesz założyć, że wprowadzone wyrażenie jest poprawne.\
Można przyjąć, że robimy matcher (sprawdzanie czy dane wejście należy do automatu utworzonego dzięki regexowi), a nie wyszukiwanie regexów - (tak zrobimy).

Pomysły:
- Traktowanie znaków i szukanie już mamy zrobione, teraz tylko zmodyfikować by tymi znakami traktowanymi literalnie były litery, cyfry i znaki.
- Dla . trzeba dodać przejście (podobne jak pozostałe), które odpowiada wszystkim znakom
- Dla znaków:
    - Dla gwiazdki dodamy dwa dodatkowe przejścia - jedno pomijające stan odpowiadający znaku i jedno powracające do tego samego stanu
    - Dla plusa dodamy jedno przejście - powracające do tego samego stanu i licznik kontrolujący liczbę przejść
    - Dla pytajnika są dwie możliwości, więc wywołamy się na jednej i drugiej.
- Dla nawiasów trzeba pamiętać i zrobić przejście do jego początku - niestety niezrobione.
- Dla klasy znaków wystarczy dodać odpowiednie znaki do przejść.
- Dla ułatwienia, kosztem złożoności, możemy matchować z backtrackingiem.

Dodatkowo zauważmy zależności między \* i +. + to 1 lub więcej powtórzeń, a * to 0 lub więcej powtórzeń, więc (x+) <=> (xx\*) .\
Zakładam, że .\* oznacza dowolne znaki 0 lub więcej powtórzeń, a nie dowolny znak(ale konkretnie 1) i jego 0 lub więcej powtórzeń.

In [3]:
lowerCase=[chr(i) for i in range(ord('a'), ord('z')+1)]
upperCase=[chr(i) for i in range(ord('A'), ord('Z')+1)]
letters=lowerCase+upperCase
digits=[chr(i+ord('0')) for i in range(0, 10)]
space=[' ']
literal = letters+digits+space
operators = {'*', '+', '?'}

In [4]:
def oneOf(pattern, i):
    assert pattern[i]=='['
    i+=1
    acc = []
    while pattern[i]!=']': # Zakladajac poprawnosc zawsze bedzie nawias zamykajacy
        acc.append(pattern[i])
        i+=1
    return acc, i
    

def groups(pattern, i):
    if pattern[i]=='.':
        return literal, i
    assert pattern[i]=='\\'
    i+=1
    acc=[]
    if pattern[i]=='d':
        acc=digits
    elif pattern[i]=='D':
        acc=letters+space
    elif pattern[i]=='w': # dla 'W' to po prostu spacja w naszym przypadku
        acc=letters+digits
    else: # zakladam poprawnosc wiec tutaj nie powinno nigdy dojsc, mozna doimplementowac wiecej grup
        acc=literal
    return acc, i


def getAcc(pattern, i):
    if pattern[i] in {'.', '\\'}:
        return groups(pattern, i)
    elif pattern[i]=='[':
        return oneOf(pattern, i)
    else:
        return [pattern[i]], i

    
def parse(pattern):
    for i in range(1, len(pattern)):
        if pattern[i] in operators:
            if pattern[i] == '+':
                if pattern[i-1]==']':
                    ix = i-2
                    while pattern[ix] != '[': # zakladam poprawnosc, wiec zawsze bedzie [
                        ix-=1
                    pattern = pattern[:i]+pattern[ix:i]+'*'+pattern[i+1:]
                elif i-2>=0 and pattern[i-2]=='\\':
                    pattern = pattern[:i]+pattern[i-2:i]+'*'+pattern[i+1:]
                else:
                    pattern = pattern[:i]+pattern[i-1]+'*'+pattern[i+1:]
    return pattern
    
    
def getop(c):
    assert c in {'*', '?'}
    return c
    
    
def build(pattern):
    pattern = parse(pattern)
    mov = [{}]
    acc, ix = getAcc(pattern, 0)
    if ix+1<len(pattern) and pattern[ix+1] in operators:
        typ = getop(pattern[ix+1])
        for i in acc:
            mov[-1][i] = (1, typ)
        ix+=1
    else:
        for i in acc:
            mov[0][i] = 1
    ix += 1
    lps = 0
    while ix<len(pattern):
        mov.append({})
        acc, ix = getAcc(pattern, ix)
        if ix+1<len(pattern) and pattern[ix+1] in operators:
            typ = getop(pattern[ix+1])
            for i in acc:
                mov[-1][i] = (len(mov), typ)
            ix+=1
        else:
            for i in acc:
                mov[-1][i] = len(mov)
        ix+=1
    return mov


def find(txt, tix, mov, mix):
    if type(mix) == tuple:
        if mix[1]=='?': # wszedlem tu bo juz sie raz cos powtorzylo, idziemy dalej
            return find(txt, tix, mov, mix[0])
        if find(txt, tix, mov, mix[0]):
            return True
        if tix<len(txt) and txt[tix] in mov[mix[0]-1]:
            return find(txt, tix+1, mov, mix) # sprawdzamy dalej, zakladajac ze ciag z * sie skonczyl lub nie
        return False
    
    if len(txt)==tix and len(mov)==mix:
        return True
    elif len(txt)==tix:
        if type(list(mov[mix].items())[0][1]) == tuple:
            return find(txt, tix, mov, mix+1)
        return False
    elif len(mov)==mix:
        return False
    
    if txt[tix] not in mov[mix]:
        if type(list(mov[mix].items())[0][1]) == tuple: # jesli byl operator *, +, ? to wszystkie elementy beda tuplami, wiec wybieram dowolny
            return find(txt, tix, mov, mix+1) # poprzedni znak sie nie powtarza
        else:
            return False
    return find(txt, tix+1, mov, mov[mix][txt[tix]])
    

def regex(source, pattern, mov=None):
    if mov is None:
        mov = build(pattern)#, alf)
    return find(source, 0, mov, 0)

Udało się zaimplementować:
- Traktowanie literalne wymaganych znaków (bardzo łatwo zmienić jakie te znaki powinny być)
- Kropka - dowolny znak
- Klasy znaków - w nawiasach kwadratowych i typu \d (dokładnie: \d, \D, \w, można łatwo dodać więcej klas)
- Specjalne znaki *, + i ?, oraz ich specjalne działanie\
Testy poniżej.

In [5]:
txt = "122 and OTH"
print(regex(txt, '\d\d\d.\w\w\w.\w\w\w')==True)
print(regex(txt, '\d\d\d.\wa\w.\w\w\w')==False)
print(regex(txt, '\d\D\d.\w.\w..\w\w')==False)
print(regex(txt, txt)==True)
print(regex(txt, '12...n\w\D[czxOsad]T[12312 Hasd]')==True)
print(regex(txt, '12...n\w\D[czxOsad]T[12312 hasd]')==False)

True
True
True
True
True
True


Jak widać matcher przechodzi testy na ., [dowolne znaki] i klasy(typu \d).

In [6]:
print(regex('aaaaaaaaa', 'a*')==True)
print(regex('aaaaaaaaa', 'a*b*a+')==True)
print(regex('aaaaaaaaa', 'a?a*')==True)
print(regex('adasdjaksdb', 'a.*b')==True)
print(regex('aaaaaaaaa', 'a?c*')==False)
print(regex('adasdjaksdb', 'a.*a?a?.*b')==True)
print(regex('', 'a?a?')==True)
print(regex('a', 'a?a?')==True)
print(regex('aa', 'a?a?')==True)
print(regex('aaa', 'a?a?')==False)
print(regex('aaa', 'a*a?')==True)
print(regex('aaaa', 'a*a')==True)

print(regex('123123', '\d+')==True)
print(regex('123123', '\d?')==False)
print(regex('123123', '\w+')==True)
print(regex('1231adsasd3', '\w+')==True)
print(regex('1231sadasd23', '\d+')==False)
print(regex('123123', '[123]+')==True)
print(regex('1234123', '[123]+')==False)
print(regex('1', '[123]?')==True)
print(regex('123', '[123]?')==False)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


Jak widać matcher przechodzi testy na \*, + i ? . Nawet w połączeniu z poprzednio sprawdzonymi opcjami(., klasy i [rozne znaki])

Podsumowując, udało się zrobić matcher sprawdzający czy cały napis pasuje do wyrażenia, umożliwiający skorzystanie z takich opcji:
- literalne traktowanie liter, cyfr i spacji
- kropki - reprezentuje dowolny znak
- operatory * - 0 lub więcej powtórzeń poprzedniego wyrażenia, + - 1 lub więcej powtórzeń, ? - 0 lub 1 powtórzenie
- klasy znaków (znaki zapisane w [], lub oznaczenia klasy typu \d, \D, \w).